# bot using kaggle data
https://www.kaggle.com/mushaya/chatbot-1

# 1)- Importing key modules

In [9]:
# support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [10]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [11]:
print(os.listdir())

['.ipynb_checkpoints', 'ai.yml', 'bot.ipynb', 'botprofile.yml', 'computers.yml', 'emotion.yml', 'food.yml', 'gossip.yml', 'greetings.yml', 'health.yml', 'history.yml', 'humor.yml', 'literature.yml', 'money.yml', 'movies.yml', 'politics.yml', 'psychology.yml', 'science.yml', 'sports.yml', 'trivia.yml']


In [16]:
questions=list()
answers=list()

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file = open(os.path.join(dirname, filename), 'rb')
        docs = yaml.safe_load(file)
        conversations = docs['conversations']
        for con in conversations:
            if len(con)>2:
                questions.append(con[0])
                replies=con[1:]
                ans=""
                for rep in replies:
                    ans+=' '+rep
                answers.append(ans)
            elif len(con)>1:
                questions.append(con[0])
                answers.append(con[1])

In [17]:
answers_with_tags=list()
for i in range( len(answers)):
    if type(answers[i])==str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers=list()
for ans in answers_with_tags:
    answers.append('<START> '+ans+' <END>')

tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions+answers)
VOCAB_SIZE= len(tokenizer.word_index)+1
print('VOCAB SIZE : {}'.format(VOCAB_SIZE))

NameError: name 'preprocessing' is not defined